# Visualizing a PPI

In [ ]:
import os
import datetime
import numpy as np

from netCDF4 import Dataset

import chart

In [ ]:
# file = os.path.expanduser('~/Downloads/data/moment-reg/20230130/PX-20230130-173637-E4.0-Z.nc')
# file = os.path.expanduser('~/Downloads/PX-20170220-050706-E2.4-Z.nc')
file = 'blob/PX-20170220-050706-E2.4-Z.nc'

with Dataset(file, mode='r') as nc:
    name = nc.getncattr('TypeName')
    elev = np.array(nc.variables['Elevation'][:], dtype=np.float32)
    azim = np.array(nc.variables['Azimuth'][:], dtype=np.float32)
    gatewidth = np.array(nc.variables['GateWidth'][:], dtype=np.float32)
    values = np.array(nc.variables[name][:], dtype=np.float32)
    values[values < -90] = np.nan
    longitude = nc.getncattr('Longitude')
    latitude = nc.getncattr('Latitude')
    sweepElev = nc.getncattr('Elevation')
    sweepTime = nc.getncattr('Time')
    symbol = file.split('.')[-2].split('-')[-1]

## Overlay for the Plot

In [ ]:
overlay = chart.atlas.Overlay((longitude, latitude))

In [ ]:
radii = np.concatenate(([1.0], np.arange(10.0, 31.0, 10.0)))
overlay.setRingRadii(radii)
overlay.load()

In [ ]:
r = 1.0e-3 * np.arange(values.shape[1]) * gatewidth[0]
a = np.deg2rad(azim)
t = datetime.datetime.utcfromtimestamp(sweepTime)
timestr = t.strftime('%Y/%m/%d %H:%M:%S')
title = f'{timestr} UTC  EL: {sweepElev:.2f}°'
symbol = 'Z'

## Chart with Overlay

In [ ]:
ppi = chart.chart.Image(a, r, values, style=symbol, overlay=overlay, title=title, figsize=(800, 600), maxrange=25.0)
ppi.image()

## Convenient Function to Populate Image

In [ ]:
def populateImage(file):
    with Dataset(file, mode='r') as nc:
        name = nc.getncattr('TypeName')
        azim = np.array(nc.variables['Azimuth'][:], dtype=np.float32)
        gatewidth = np.array(nc.variables['GateWidth'][:], dtype=np.float32)
        values = np.array(nc.variables[name][:], dtype=np.float32)
        values[values < -90] = np.nan
        r = 1.0e-3 * np.arange(values.shape[1]) * gatewidth[0]
        a = np.deg2rad(azim)
        symbol = file.split('.')[-2].split('-')[-1]
    ppi.set_data(values, style=symbol, a=a, r=r)

## More Images

In [ ]:
populateImage(os.path.expanduser('blob/PX-20170220-050706-E2.4-V.nc'))
ppi.image()

In [ ]:
populateImage(os.path.expanduser('blob/PX-20170220-050706-E2.4-D.nc'))
ppi.image()

In [ ]:
populateImage(os.path.expanduser('blob/PX-20170220-050706-E2.4-R.nc'))
ppi.image()